## H2O-3のk分割交差検証法の仕様に関して

[Document: Cross-Validation](https://docs.h2o.ai/h2o/latest-stable/h2o-docs/cross-validation.html)

In [1]:
from ordered_set import OrderedSet

from h2o.estimators import H2ORandomForestEstimator
import h2o
print(h2o.__version__)

3.36.0.4


In [2]:
h2o.init()

Checking whether there is an H2O instance running at http://localhost:54321 ..... not found.
Attempting to start a local H2O server...
  Java Version: java version "11.0.9" 2020-10-20 LTS; Java(TM) SE Runtime Environment 18.9 (build 11.0.9+7-LTS); Java HotSpot(TM) 64-Bit Server VM 18.9 (build 11.0.9+7-LTS, mixed mode)
  Starting server from /opt/anaconda3/lib/python3.8/site-packages/h2o/backend/bin/h2o.jar
  Ice root: /var/folders/50/glkvvhjx03g22fwy2r0tgylr0000gn/T/tmpiojqcf7i
  JVM stdout: /var/folders/50/glkvvhjx03g22fwy2r0tgylr0000gn/T/tmpiojqcf7i/h2o_YShimada_MBP16_started_from_python.out
  JVM stderr: /var/folders/50/glkvvhjx03g22fwy2r0tgylr0000gn/T/tmpiojqcf7i/h2o_YShimada_MBP16_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321 ... successful.


H2O_cluster_uptime:,01 secs
H2O_cluster_timezone:,Asia/Tokyo
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.36.0.4
H2O_cluster_version_age:,3 months and 22 days !!!
H2O_cluster_name:,H2O_from_python_YShimada_MBP16_7v6iqx
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,8 Gb
H2O_cluster_total_cores:,16
H2O_cluster_allowed_cores:,16
H2O_cluster_status:,"locked, healthy"


In [3]:
frame_CreditCard = h2o.import_file('https://h2oai-jpn-public.s3.amazonaws.com/sample-data/uci-credit-card/UCI_Credit_Card3.csv',
                                  col_types={'default_payment_next_month':'enum'})
frame_CreditCard.shape

Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%


(30000, 25)

In [4]:
frame_CreditCard.describe()

Rows:30000
Cols:25




,ID,LIMIT_BAL,SEX,EDUCATION,MARRIAGE,AGE,PAY_1,PAY_2,PAY_3,PAY_4,PAY_5,PAY_6,BILL_AMT1,BILL_AMT2,BILL_AMT3,BILL_AMT4,BILL_AMT5,BILL_AMT6,PAY_AMT1,PAY_AMT2,PAY_AMT3,PAY_AMT4,PAY_AMT5,PAY_AMT6,default_payment_next_month
type,int,int,enum,enum,enum,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,enum
mins,1.0,10000.0,,,,21.0,-2.0,-2.0,-2.0,-2.0,-2.0,-2.0,-165580.0,-69777.0,-157264.0,-170000.0,-81334.0,-339603.0,0.0,0.0,0.0,0.0,0.0,0.0,
mean,15000.5,167484.32266666667,,,,35.4855,-0.01669999999999999,-0.13376666666666667,-0.1662,-0.2206666666666667,-0.2662,-0.2911,51223.3309,49179.07516666667,47013.1548,43262.94896666666,40311.40096666667,38871.7604,5663.5805,5921.163499999999,5225.6815,4826.076866666666,4799.387633333334,5215.502566666667,
maxs,30000.0,1000000.0,,,,79.0,8.0,8.0,8.0,8.0,8.0,8.0,964511.0,983931.0,1664089.0,891586.0,927171.0,961664.0,873552.0,1684259.0,896040.0,621000.0,426529.0,528666.0,
sigma,8660.398374208891,129747.6615672024,,,,9.217904068090185,1.1238015279973348,1.1971859730345533,1.1968675684465737,1.1691386224023375,1.1331874060027483,1.1499876256079027,73635.86057552956,71173.76878252834,69349.38742703684,64332.85613391631,60797.15577026487,59554.10753674573,16563.280354025766,23040.870402057233,17606.961469803104,15666.159744032007,15278.305679144789,17777.465775435296,
zeros,0,0,,,,0,14737,15730,15764,16455,16947,16286,2008,2506,2870,3195,3506,4020,5249,5396,5968,6408,6703,7173,
missing,0,0,0,345,54,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
0,1.0,20000.0,M,university,married,24.0,2.0,2.0,-1.0,-1.0,-2.0,-2.0,3913.0,3102.0,689.0,0.0,0.0,0.0,0.0,689.0,0.0,0.0,0.0,0.0,1
1,2.0,120000.0,M,university,single,26.0,-1.0,2.0,0.0,0.0,0.0,2.0,2682.0,1725.0,2682.0,3272.0,3455.0,3261.0,0.0,1000.0,1000.0,1000.0,0.0,2000.0,1
2,3.0,90000.0,M,university,single,34.0,0.0,0.0,0.0,0.0,0.0,0.0,29239.0,14027.0,13559.0,14331.0,14948.0,15549.0,1518.0,1500.0,1000.0,1000.0,1000.0,5000.0,0


In [5]:
y_col_creditc = 'default_payment_next_month'
print('ターゲット変数： {}'.format(y_col_creditc))
X_cols_creditc = list( OrderedSet(frame_CreditCard.names) - OrderedSet(['ID',y_col_creditc]) )    # IDとターゲット変数を除外
print('特徴量： {}'.format(X_cols_creditc))

ターゲット変数： default_payment_next_month
特徴量： ['LIMIT_BAL', 'SEX', 'EDUCATION', 'MARRIAGE', 'AGE', 'PAY_1', 'PAY_2', 'PAY_3', 'PAY_4', 'PAY_5', 'PAY_6', 'BILL_AMT1', 'BILL_AMT2', 'BILL_AMT3', 'BILL_AMT4', 'BILL_AMT5', 'BILL_AMT6', 'PAY_AMT1', 'PAY_AMT2', 'PAY_AMT3', 'PAY_AMT4', 'PAY_AMT5', 'PAY_AMT6']


In [6]:
rf = H2ORandomForestEstimator(seed=42, model_id='default_rf', 
                              nfolds=3,       # k分割交差検証（k=3）
                              keep_cross_validation_fold_assignment=True,    # k分割交差検証結果の保持
                              keep_cross_validation_models=True,       # k分割交差検証結果の保持 
                              keep_cross_validation_predictions=True,     # k分割交差検証結果の保持
                             )

rf.train(x=X_cols_creditc, y=y_col_creditc, training_frame=frame_CreditCard)

drf Model Build progress: |██████████████████████████████████████████████████████| (done) 100%
Model Details
H2ORandomForestEstimator :  Distributed Random Forest
Model Key:  default_rf


Model Summary: 


,,number_of_trees,number_of_internal_trees,model_size_in_bytes,min_depth,max_depth,mean_depth,min_leaves,max_leaves,mean_leaves
0,,50.0,50.0,1763773.0,20.0,20.0,20.0,2329.0,3365.0,2786.96




ModelMetricsBinomial: drf
** Reported on train data. **

MSE: 0.13917423251450023
RMSE: 0.3730606284700923
LogLoss: 0.45137414871644
Mean Per-Class Error: 0.3004394173134787
AUC: 0.7604702742012333
AUCPR: 0.532387037140067
Gini: 0.5209405484024665

Confusion Matrix (Act/Pred) for max f1 @ threshold = 0.28720455889143964: 


,,0,1,Error,Rate
0,0,19472.0,3892.0,0.1666,(3892.0/23364.0)
1,1,2882.0,3754.0,0.4343,(2882.0/6636.0)
2,Total,22354.0,7646.0,0.2258,(6774.0/30000.0)



Maximum Metrics: Maximum metrics at their respective thresholds


...


Scoring History: 


,,timestamp,duration,number_of_trees,training_rmse,training_logloss,training_auc,training_pr_auc,training_lift,training_classification_error
0,,2022-07-23 09:34:52,13.348 sec,0.0,NaN,NaN,NaN,NaN,NaN,NaN
1,,2022-07-23 09:34:53,13.480 sec,1.0,0.465341,5.381253,0.635312,0.346939,1.990726,0.282741
2,,2022-07-23 09:34:53,13.558 sec,2.0,0.457950,4.924225,0.635771,0.358521,2.111078,0.268976
3,,2022-07-23 09:34:53,13.641 sec,3.0,0.446898,4.206738,0.646121,0.372270,2.220843,0.287456
4,,2022-07-23 09:34:53,13.728 sec,4.0,0.435198,3.481075,0.661315,0.384883,2.237912,0.263200
5,,2022-07-23 09:34:53,13.812 sec,5.0,0.427128,2.981562,0.671441,0.395788,2.297721,0.269986
6,,2022-07-23 09:34:53,13.898 sec,6.0,0.421275,2.591609,0.676691,0.404879,2.386422,0.265179
7,,2022-07-23 09:34:53,13.981 sec,7.0,0.414021,2.172954,0.687001,0.419448,2.469786,0.270792
8,,2022-07-23 09:34:53,14.069 sec,8.0,0.408464,1.847199,0.696452,0.432431,2.596639,0.259348
9,,2022-07-23 09:34:53,14.157 sec,9.0,0.404249,1.614503,0.704230,0.441587,2.665414,0.261137



See the whole table with table.as_data_frame()

Variable Importances: 


,variable,relative_importance,scaled_importance,percentage
0,PAY_1,17112.259766,1.000000,0.146720
1,PAY_2,9487.572266,0.554431,0.081346
2,AGE,7630.824219,0.445927,0.065426
3,LIMIT_BAL,7025.684082,0.410564,0.060238
4,PAY_5,6361.038086,0.371724,0.054539
5,PAY_3,6144.964355,0.359097,0.052687
6,BILL_AMT1,5482.916504,0.320409,0.047010
7,PAY_4,4847.465820,0.283274,0.041562
8,BILL_AMT2,4568.901855,0.266996,0.039174
9,BILL_AMT5,4330.010742,0.253036,0.037125



See the whole table with table.as_data_frame()


k=3と指定したので４つのモデルが作成されている  
各k分割に対応したモデルが３つ、全データで推定したモデルが１つ

train dataとcross-validation dataに対する結果がそれぞれ表示される  
Confusion Matrixを確認すると、train data、cross-validation data共にオブザベーション数が30,000となっている  
train dataの結果は、全データで推定したモデル（メインモデル）の全データに対する結果  
cross-validation dataの結果は、各k分割に対応した３つのモデルの予測結果を縦結合して、正解と比較したもの（３つのモデルの結果の平均ではない）

### k分割交差検証結果の確認

cross_validation_fold_assignmentメソッドでFold IDが確認できる（[keep_cross_validation_fold_assignment=Trueとした場合](https://docs.h2o.ai/h2o/latest-stable/h2o-docs/data-science/algo-params/keep_cross_validation_fold_assignment.html)）

In [7]:
rf.cross_validation_fold_assignment().concat(frame_CreditCard)

fold_assignment,ID,LIMIT_BAL,SEX,EDUCATION,MARRIAGE,AGE,PAY_1,PAY_2,PAY_3,PAY_4,PAY_5,PAY_6,BILL_AMT1,BILL_AMT2,BILL_AMT3,BILL_AMT4,BILL_AMT5,BILL_AMT6,PAY_AMT1,PAY_AMT2,PAY_AMT3,PAY_AMT4,PAY_AMT5,PAY_AMT6,default_payment_next_month
0,1,20000,M,university,married,24,2,2,-1,-1,-2,-2,3913,3102,689,0,0,0,0,689,0,0,0,0,1
2,2,120000,M,university,single,26,-1,2,0,0,0,2,2682,1725,2682,3272,3455,3261,0,1000,1000,1000,0,2000,1
1,3,90000,M,university,single,34,0,0,0,0,0,0,29239,14027,13559,14331,14948,15549,1518,1500,1000,1000,1000,5000,0
0,4,50000,M,university,married,37,0,0,0,0,0,0,46990,48233,49291,28314,28959,29547,2000,2019,1200,1100,1069,1000,0
1,5,50000,F,university,married,57,-1,0,-1,0,0,0,8617,5670,35835,20940,19146,19131,2000,36681,10000,9000,689,679,0
2,6,50000,F,graduate school,single,37,0,0,0,0,0,0,64400,57069,57608,19394,19619,20024,2500,1815,657,1000,1000,800,0
1,7,500000,F,graduate school,single,29,0,0,0,0,0,0,367965,412023,445007,542653,483003,473944,55000,40000,38000,20239,13750,13770,0
0,8,100000,M,university,single,23,0,-1,-1,0,0,-1,11876,380,601,221,-159,567,380,601,0,581,1687,1542,0
1,9,140000,M,high school,married,28,0,0,2,0,0,0,11285,14096,12108,12211,11793,3719,3329,0,432,1000,1000,1000,0
1,10,20000,F,high school,single,35,-2,-2,-2,-2,-1,-1,0,0,0,0,13007,13912,0,0,0,13007,1122,0,0


In [8]:
rf.cross_validation_fold_assignment().unique()

# 0,1,2の3水準

C1
0
2
1


cross_validation_modelsメソッドを実行すると、k分割交差検証法（k=3）で作成した３つのモデルの結果が確認できる（[keep_cross_validation_models=Trueとした場合](https://docs.h2o.ai/h2o/latest-stable/h2o-docs/data-science/algo-params/keep_cross_validation_models.html)）

In [9]:
rf.cross_validation_models()

Model Details
H2ORandomForestEstimator :  Distributed Random Forest
Model Key:  default_rf_cv_1


Model Summary: 


,,number_of_trees,number_of_internal_trees,model_size_in_bytes,min_depth,max_depth,mean_depth,min_leaves,max_leaves,mean_leaves
0,,50.0,50.0,1321645.0,20.0,20.0,20.0,1796.0,2333.0,2086.74




ModelMetricsBinomial: drf
** Reported on train data. **

MSE: 0.1400935546216046
RMSE: 0.37429073542048114
LogLoss: 0.4637632427506957
Mean Per-Class Error: 0.30587996798113487
AUC: 0.7579391691436009
AUCPR: 0.5245900783517689
Gini: 0.5158783382872019

Confusion Matrix (Act/Pred) for max f1 @ threshold = 0.34264897961153956: 


,,0,1,Error,Rate
0,0,13606.0,1923.0,0.1238,(1923.0/15529.0)
1,1,2142.0,2248.0,0.4879,(2142.0/4390.0)
2,Total,15748.0,4171.0,0.2041,(4065.0/19919.0)



Maximum Metrics: Maximum metrics at their respective thresholds


...

,,timestamp,duration,number_of_trees,training_rmse,training_logloss,training_auc,training_pr_auc,training_lift,training_classification_error,validation_rmse,validation_logloss,validation_auc,validation_pr_auc,validation_lift,validation_classification_error
0,,2022-07-23 09:34:48,9.171 sec,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,,2022-07-23 09:34:48,9.225 sec,1.0,0.479416,6.644748,0.623153,0.353765,2.122673,0.258257,0.489794,6.997610,0.613718,0.331847,1.950799,0.264416
2,,2022-07-23 09:34:48,9.270 sec,2.0,0.469411,5.852792,0.623178,0.356168,2.155594,0.254737,0.431011,3.114699,0.665673,0.393046,2.631728,0.309725
3,,2022-07-23 09:34:48,9.315 sec,3.0,0.457993,5.071756,0.633295,0.365927,2.230332,0.275387,0.408272,1.802494,0.692836,0.434925,3.073545,0.324292
4,,2022-07-23 09:34:48,9.368 sec,4.0,0.447161,4.292752,0.645519,0.376885,2.297420,0.277946,0.397997,1.207423,0.709600,0.456152,3.283736,0.228451
5,,2022-07-23 09:34:48,9.448 sec,5.0,0.438548,3.659960,0.655180,0.382942,2.319352,0.298055,0.392556,0.961099,0.720315,0.467427,3.222557,0.235885
6,,2022-07-23 09:34:49,9.506 sec,6.0,0.430278,3.107544,0.666082,0.394317,2.403829,0.303491,0.388291,0.791645,0.729691,0.478229,3.248443,0.247840
7,,2022-07-23 09:34:49,9.559 sec,7.0,0.420919,2.533261,0.681628,0.410253,2.525224,0.302004,0.385439,0.672047,0.735347,0.486077,3.364399,0.268535
8,,2022-07-23 09:34:49,9.620 sec,8.0,0.414781,2.182299,0.688922,0.420693,2.598629,0.302453,0.383230,0.603691,0.739973,0.493886,3.431033,0.222323
9,,2022-07-23 09:34:49,9.687 sec,9.0,0.410424,1.911376,0.698677,0.428492,2.604483,0.304167,0.381001,0.572858,0.744547,0.501245,3.375833,0.224834



See the whole table with table.as_data_frame()

Variable Importances: 


,variable,relative_importance,scaled_importance,percentage
0,PAY_1,12090.763672,1.000000,0.146562
1,PAY_2,6286.490723,0.519942,0.076203
2,AGE,5478.680176,0.453129,0.066411
3,LIMIT_BAL,4937.300293,0.408353,0.059849
4,PAY_5,4246.284668,0.351201,0.051473
5,BILL_AMT1,3989.275146,0.329944,0.048357
6,PAY_3,3957.828125,0.327343,0.047976
7,BILL_AMT2,3347.857666,0.276894,0.040582
8,BILL_AMT4,3182.572021,0.263223,0.038578
9,BILL_AMT6,3130.163086,0.258889,0.037943



See the whole table with table.as_data_frame()


[, , ]

Confusion Matrixを確認すると、オブザベーション数が、train data : cross-validation data = 2 : 1となっている

インデックス指定で、各モデルの取得

In [10]:
print(rf.model_id)
print(rf.cross_validation_models()[0].model_id)
print(rf.cross_validation_models()[1].model_id)
print(rf.cross_validation_models()[2].model_id)

# 各k分割交差検証のモデルは、「本モデルID_cv_1」といった形でIDが振られる

default_rf
default_rf_cv_1
default_rf_cv_2
default_rf_cv_3


In [11]:
# 検証データに対するAUCの確認
print(rf.auc(xval=True))   # メインモデルのk分割交差検証結果はxvalで指定
print(rf.cross_validation_models()[0].auc(valid=True))
print(rf.cross_validation_models()[1].auc(valid=True))
print(rf.cross_validation_models()[2].auc(valid=True))

0.7707114288387084
0.7701965800649073
0.7673221115517747
0.7748595260603883


In [12]:
# default_rf_cv_1モデルの確認
rf.cross_validation_models()[0]

Model Details
H2ORandomForestEstimator :  Distributed Random Forest
Model Key:  default_rf_cv_1


Model Summary: 


,,number_of_trees,number_of_internal_trees,model_size_in_bytes,min_depth,max_depth,mean_depth,min_leaves,max_leaves,mean_leaves
0,,50.0,50.0,1321645.0,20.0,20.0,20.0,1796.0,2333.0,2086.74




ModelMetricsBinomial: drf
** Reported on train data. **

MSE: 0.1400935546216046
RMSE: 0.37429073542048114
LogLoss: 0.4637632427506957
Mean Per-Class Error: 0.30587996798113487
AUC: 0.7579391691436009
AUCPR: 0.5245900783517689
Gini: 0.5158783382872019

Confusion Matrix (Act/Pred) for max f1 @ threshold = 0.34264897961153956: 


,,0,1,Error,Rate
0,0,13606.0,1923.0,0.1238,(1923.0/15529.0)
1,1,2142.0,2248.0,0.4879,(2142.0/4390.0)
2,Total,15748.0,4171.0,0.2041,(4065.0/19919.0)



Maximum Metrics: Maximum metrics at their respective thresholds


...



Scoring History: 


,,timestamp,duration,number_of_trees,training_rmse,training_logloss,training_auc,training_pr_auc,training_lift,training_classification_error,validation_rmse,validation_logloss,validation_auc,validation_pr_auc,validation_lift,validation_classification_error
0,,2022-07-23 09:34:39,0.114 sec,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,,2022-07-23 09:34:40,0.584 sec,1.0,0.475407,6.262905,0.639114,0.343703,1.923618,0.270059,0.483177,6.505083,0.629903,0.337248,1.853792,0.285388
2,,2022-07-23 09:34:40,0.843 sec,2.0,0.461373,5.213616,0.642780,0.360119,2.078157,0.275622,0.425526,2.489662,0.681250,0.409225,2.647237,0.311973
3,,2022-07-23 09:34:40,1.179 sec,3.0,0.453144,4.530679,0.644189,0.364427,2.136211,0.283089,0.408262,1.590285,0.700436,0.439311,2.899988,0.333697
4,,2022-07-23 09:34:41,1.567 sec,4.0,0.441300,3.802571,0.656550,0.378150,2.215298,0.275887,0.395586,1.041950,0.721921,0.469024,3.138758,0.225870
5,,2022-07-23 09:34:41,1.864 sec,5.0,0.434323,3.301880,0.663409,0.383704,2.252357,0.282410,0.391098,0.821308,0.731944,0.475290,3.066349,0.238468
6,,2022-07-23 09:34:41,2.083 sec,6.0,0.427302,2.869675,0.670140,0.394782,2.346020,0.286020,0.387413,0.704124,0.739763,0.484368,3.006808,0.265053
7,,2022-07-23 09:34:41,2.292 sec,7.0,0.418164,2.387256,0.683899,0.412701,2.502316,0.267575,0.385521,0.609003,0.742649,0.491519,3.155229,0.237179
8,,2022-07-23 09:34:42,2.467 sec,8.0,0.413918,2.117603,0.688672,0.420298,2.570499,0.266262,0.383440,0.567393,0.744187,0.498689,3.332988,0.234203
9,,2022-07-23 09:34:42,2.602 sec,9.0,0.409710,1.850136,0.696921,0.428375,2.626289,0.263987,0.381717,0.540431,0.748634,0.504792,3.421868,0.237278



See the whole table with table.as_data_frame()

Variable Importances: 


,variable,relative_importance,scaled_importance,percentage
0,PAY_1,10670.290039,1.000000,0.128914
1,PAY_2,6514.335938,0.610512,0.078703
2,AGE,5659.385742,0.530387,0.068374
3,LIMIT_BAL,5127.861328,0.480574,0.061953
4,PAY_5,4691.908691,0.439717,0.056686
5,PAY_3,4537.472656,0.425244,0.054820
6,BILL_AMT1,3813.604492,0.357404,0.046074
7,PAY_4,3425.384033,0.321021,0.041384
8,BILL_AMT2,3320.691406,0.311209,0.040119
9,BILL_AMT6,3207.783203,0.300628,0.038755



See the whole table with table.as_data_frame()


In [13]:
type(rf.cross_validation_models()[1])

h2o.estimators.random_forest.H2ORandomForestEstimator

In [14]:
# default_rf_cv_1モデルでの予測
rf.cross_validation_models()[1].predict(frame_CreditCard)

drf prediction progress: |███████████████████████████████████████████████████████| (done) 100%


predict,p0,p1
1,0.125954,0.874046
1,0.230269,0.769731
0,0.921055,0.078945
0,0.865383,0.134617
0,0.83701,0.16299
0,0.875215,0.124785
0,0.83,0.17
0,0.880318,0.119682
0,0.807654,0.192346
0,0.863379,0.136621


cross_validation_holdout_predictionsメソッドで、k分割交差検証の検証データに対する予測を取得（[keep_cross_validation_predictions=Trueとした場合](https://docs.h2o.ai/h2o/latest-stable/h2o-docs/data-science/algo-params/keep_cross_validation_predictions.html)）

In [15]:
rf.cross_validation_fold_assignment().concat( rf.cross_validation_holdout_predictions() )

fold_assignment,predict,p0,p1
0,1,0.523333,0.476667
2,1,0.69117,0.30883
1,0,0.921055,0.078945
0,0,0.859289,0.140711
1,0,0.83701,0.16299
2,0,0.83336,0.16664
1,0,0.83,0.17
0,0,0.71998,0.28002
1,0,0.807654,0.192346
1,0,0.863379,0.136621


In [16]:
# 学習データに対するメインモデルの予測
rf.predict(frame_CreditCard)

drf prediction progress: |███████████████████████████████████████████████████████| (done) 100%


predict,p0,p1
1,0.254993,0.745007
1,0.240469,0.759531
0,0.919208,0.0807917
0,0.856847,0.143153
0,0.904236,0.0957645
0,0.899109,0.100891
0,0.977647,0.0223529
0,0.926666,0.0733342
0,0.820658,0.179342
0,0.984018,0.015982


In [17]:
rf.cross_validation_holdout_predictions().shape

(30000, 3)

In [18]:
#rf.cross_validation_predictions()

各k分割交差検証モデルの結果

In [19]:
rf.cross_validation_metrics_summary()


Cross-Validation Metrics Summary: 


,,mean,sd,cv_1_valid,cv_2_valid,cv_3_valid
0,accuracy,0.788678,0.012310,0.802599,0.779227,0.784207
1,auc,0.770793,0.003804,0.770197,0.767322,0.774860
2,err,0.211322,0.012310,0.197401,0.220773,0.215793
3,err_count,2112.666700,109.367880,1990.000000,2200.000000,2148.000000
4,f0point5,0.528634,0.020526,0.551378,0.511489,0.523033
5,f1,0.537093,0.006418,0.534829,0.532114,0.544336
6,f2,0.547055,0.024943,0.519245,0.554472,0.567448
7,lift_top_group,3.665621,0.170844,3.555187,3.862403,3.579272
8,logloss,0.436338,0.002979,0.437850,0.438258,0.432906
9,max_per_class_error,0.445407,0.039761,0.490650,0.429549,0.416022


In [20]:
h2o.shutdown()

H2O session _sid_920e closed.


<ipython-input-20-1edf85295eae>:1: H2ODeprecationWarning: Deprecated, use ``h2o.cluster().shutdown()``.
  h2o.shutdown()
